# 심층 신경망 (파이토치)

<table align="left"><tr><td>
<a href="https://colab.research.google.com/github/rickiepark/hg-mldl2/blob/main/07-2-pt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="코랩에서 실행하기"/></a>
</td></tr></table>

In [1]:
import torch

torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed(42)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [2]:
from torchvision.datasets import FashionMNIST

# 패션 MNIST 데이터셋 다운로드
train = FashionMNIST(root='.', train=True, download=True)
test = FashionMNIST(root='.', train=False, download=True)

In [3]:
type(train.data)

torch.Tensor

In [4]:
print(train.data.shape, test.data.shape)

torch.Size([60000, 28, 28]) torch.Size([10000, 28, 28])


In [5]:
print(train.targets.shape, test.targets.shape)

torch.Size([60000]) torch.Size([10000])


In [6]:
train_input = train.data
train_target = train.targets

In [7]:
train_scaled = train_input / 255.0

In [8]:
from sklearn.model_selection import train_test_split

train_scaled, val_scaled, train_target, val_target = train_test_split(
    train_scaled, train_target, test_size=0.2, random_state=42)

In [9]:
print(train_scaled.shape, val_scaled.shape)

torch.Size([48000, 28, 28]) torch.Size([12000, 28, 28])


In [10]:
import torch
import torch.nn as nn

model = nn.Sequential(
    nn.Flatten(),
    nn.Linear(784, 100),
    nn.ReLU(),
    nn.Linear(100, 10)
)

In [11]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

epochs = 5
batches = int(len(train_scaled)/32)
for epoch in range(epochs):
    model.train()
    epoch_loss = 0
    for i in range(batches):
        inputs = train_scaled[i*32:(i+1)*32]
        targets = train_target[i*32:(i+1)*32]
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    print(f"에포크:{epoch + 1}, 손실:{epoch_loss/batches:.4f}")

에포크:1, 손실:0.5428
에포크:2, 손실:0.4004
에포크:3, 손실:0.3594
에포크:4, 손실:0.3320
에포크:5, 손실:0.3118


In [12]:
model.eval()
with torch.no_grad():
    output = model(val_scaled)
    predicted = torch.argmax(output, 1)
    corrects = (predicted == val_target).sum().item()

accuracy = corrects / len(val_target)
print(f"검증 정확도: {accuracy:.4f}")

검증 정확도: 0.8734
